## Approach

CNN with 2 frames

## Setup

In [1]:
import random
from collections import OrderedDict
from pathlib import Path
from PIL import Image
from efficientnet_pytorch import EfficientNet

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler, random_split, Subset
from torchvision import transforms
import pytorch_lightning as pl

import wandb
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping

pl.seed_everything(hash("setting random seeds") % 2**32 - 1)

/home/sharif/anaconda3/lib/python3.8/site-packages/wandb/util.py:36: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.9 it will stop working
  from collections import namedtuple, Mapping, Sequence
/home/sharif/anaconda3/lib/python3.8/site-packages/wandb/vendor/graphql-core-1.1/graphql/type/directives.py:55: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.9 it will stop working
  assert isinstance(locations, collections.Iterable), 'Must provide locations for directive.'


181269041

In [2]:
wandb.login()

/home/sharif/.local/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
wandb: Currently logged in as: sharif (use `wandb login --relogin` to force relogin)


True

In [3]:
p = {
    'project_name': 'vehicle-speed-estimation',
    'batch_size': 32,
    'w': 240, #240 480
    'h': 320, #320 640
    'model': 'efficientnet-b0',
    'split': 0.2,
    'mean': .1,
    'std': .5,
    'divide_y': 10
}

In [4]:
IMAGES = '/home/sharif/Documents/Challenges/commai-challenge/data/frames/train'
LABELS = '/home/sharif/Documents/Challenges/commai-challenge/data/labels/train.txt'

In [5]:
tfms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((p['w'],p['h'])),
    transforms.Normalize((p['mean'],p['mean'],p['mean']),(p['std'],p['std'],p['std']))
])

## Dataset

In [6]:
class DS(Dataset):
    def __init__(self, images, labels):
        self.images = Path(images)
        self.labels = open(labels).readlines()
        self.n_images = len(list(self.images.glob('*.png')))
    def __len__(self): return self.n_images

    def __getitem__(self, idx):
        'Returns a random batch of len `seq`'
        if idx != 0: idx -= 1
        f1 = f'{self.images}/{str(idx)}.png'
        f2 = f'{self.images}/{str(idx+1)}.png'
        image1 = Image.open(f1)
        image2 = Image.open(f2)
        img1 = tfms(image1)
        img2 = tfms(image2)
        x = torch.cat((img1, img2))
        y = float(self.labels[idx+1].split()[0])
        return x, torch.Tensor([y])

In [7]:
ds = DS(IMAGES, LABELS)

In [8]:
train_idx = int(len(ds) * (1-p['split']))
valid_idx = int(len(ds) * p['split'])
train_idx, valid_idx

(16320, 4080)

In [9]:
train_ds = Subset(ds, list(range(train_idx)))
valid_ds = Subset(ds, list(range(train_idx, train_idx+valid_idx)))

In [10]:
#train_ds, valid_ds = random_split(ds, [train_idx, valid_idx])

In [11]:
len(train_ds), len(valid_ds)

(16320, 4080)

In [12]:
x,y = ds[11]

assert len(ds) > 10
assert list(y.shape) == [1]
assert list(x.shape) == [6,p['w'],p['h']]

In [13]:
x.mean(), x.std()

(tensor(-0.0451), tensor(0.2354))

## Model

In [14]:
class Model(pl.LightningModule):
    def __init__(self, p):
        super().__init__()
        self.hparams = p
        self.save_hyperparameters()
        
        self.en = EfficientNet.from_pretrained(p['model'], in_channels=6, num_classes=1)
        
    def forward(self, x):
        return self.en(x)
    
    def step(self, batch, batch_idx):
        x,y = batch
        y /= p['divide_y']
        y_hat = self(x)
        loss = F.mse_loss(y_hat, y)
        abs_loss = torch.abs(y_hat-y).mean().sum()
        return OrderedDict({
            'loss': loss,
            'abs_loss': abs_loss
        })
        
    def training_step(self, batch, batch_idx):
        out = self.step(batch, batch_idx)
        self.log('train_batch_loss', out['loss'])
        self.log('train_batch_abs_loss', out['abs_loss'])
        return out
    
    def training_epoch_end(self, outputs):
        loss = torch.stack([output['loss'] for output in outputs]).float().mean()
        abs_loss = torch.stack([output['abs_loss'] for output in outputs]).float().mean()
        self.log('train_loss', loss)
        self.log('train_abs_loss', abs_loss)
    
    def validation_step(self, batch, batch_idx):
        out = self.step(batch, batch_idx)
        self.log('val_batch_loss', out['loss'])
        self.log('val_batch_abs_loss', out['abs_loss'])
        return out
    
    def validation_epoch_end(self, outputs):
        loss = torch.stack([output['loss'] for output in outputs]).float().mean()
        abs_loss = torch.stack([output['abs_loss'] for output in outputs]).float().mean()
        self.log('val_abs_loss', abs_loss)
        self.log('val_loss', loss)
    
    def configure_optimizers(self): return optim.Adam(self.parameters())

In [15]:
train_dl = DataLoader(train_ds, shuffle=True, batch_size=p['batch_size'], num_workers=6, pin_memory=True)
valid_dl = DataLoader(valid_ds, shuffle=True, batch_size=p['batch_size'], num_workers=6, pin_memory=True)

In [16]:
wandb_logger = WandbLogger(project="vehicle-speed-estimation")

In [17]:
checkpoint = ModelCheckpoint(
    monitor='val_loss',
    filename='cnn-{epoch:02d}-{val_loss:.4f}',
    save_top_k=2,
    mode='min'
)

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,
    verbose=True,
    mode='min'
)

In [18]:
trainer = pl.Trainer(gpus=1,
                     fast_dev_run=False,
                     log_every_n_steps=10,
                     logger=wandb_logger,
                     #overfit_batches=5,
                     deterministic=True,
                     callbacks=[checkpoint,early_stopping]
                    )

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


In [19]:
m = Model(p)
trainer.fit(m, train_dl, valid_dl)

Loaded pretrained weights for efficientnet-b0



  | Name | Type         | Params
--------------------------------------
0 | en   | EfficientNet | 4 M   
/home/sharif/anaconda3/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:45: UserWarning: Your val_dataloader has `shuffle=True`, it is best practice to turn this off for validation and test dataloaders.
  warnings.warn(*args, **kwargs)


1

In [ ]:
wandb.finish()

In [ ]:
#ckp = '/home/sharif/Documents/Challenges/vehicle-speed-estimation-v2/vehicle-speed-estimation/22fcqxam/checkpoints/cnn-epoch=02-val_loss=0.8512.ckpt'

In [ ]:
m = Model(p)
m = m.load_from_checkpoint(ckp);
m.eval();

In [ ]:
x,y = valid_ds[190]
print(y)
x.shape
y_hat = m(x.unsqueeze(0))
print(y_hat)

In [ ]:
1/32*(y_hat - y)**2

In [ ]:
def inference(f1, f2):
    image1 = Image.open(f1)
    image2 = Image.open(f2)
    img1 = tfms(image1)
    img2 = tfms(image2)
    x = torch.cat((img1, img2)).unsqueeze(0)
    return m(x)

In [ ]:
f1 = '/home/sharif/Documents/Challenges/commai-challenge/data/frames/test/3000.png'
f2 = '/home/sharif/Documents/Challenges/commai-challenge/data/frames/test/3001.png'

In [ ]:
inference(f1,f2)